In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import glob #경로를 알아내는 라이브러리
glob.glob("/kaggle/input/data-zips/train/train/*")

In [ ]:
!unzip /kaggle/input/invasive-species-monitoring/train_labels.csv.zip

In [ ]:
train = pd.read_csv("train_labels.csv")

In [ ]:
train

In [ ]:
train["invasive"] = train["invasive"].astype(str)


In [ ]:
train["name"] = train["name"].apply(lambda x:"/kaggle/input/data-zips/train/train/"+ str(x) +".jpg")

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid = train_test_split(train, test_size=0.2, random_state=42, stratify=train["invasive"])

In [ ]:
test = pd.DataFrame({"name":glob.glob("/kaggle/input/data-zips/test/test/*")}) 

In [ ]:
test #data aggmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
idg = ImageDataGenerator(horizontal_flip=True,) #shift누르고 tab다섯번 누르면 함수 정보 볼 수 있다. image aggmentation쓸 때는 epoch을 늘려 다양한 정보를 학습할 수 있도록
train_generator = idg.flow_from_dataframe(X_train,x_col="name",y_col='invasive',target_size=(300,300),batch_size=50)#ram떄문에 끊어서 학습
valid_generator = idg.flow_from_dataframe(X_valid,x_col="name",y_col='invasive',target_size=(300,300),batch_size=50)
test_generator = idg.flow_from_dataframe(test,x_col='name',y_col=None, target_size=(300,300), batch_size=50,class_mode=None, shuffle=False)  #정답값이 없기 떄문에 이렇게 한다.

In [ ]:
train_generator

In [ ]:
!pip install -U git+https://github.com/qubvel/efficientnet

from keras.models import Sequential
from keras.layers import *
from keras.applications.inception_v3 import InceptionV3
from efficientnet.tfkeras import EfficientNetB1
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
model = Sequential() 
model.add(EfficientNetB1(weights="imagenet",include_top=False,pooling="avg")) #weight : 다른 곳에서 이미 학습했던 weight를 먼저 가지고 옴(전이 학습) , pooling="avg"는 flatten을 대신해서 정보 손실을 감소
model.add(Dense(2,activation="softmax"))
model.compile(metrics=["acc"],loss="categorical_crossentropy",optimizer="adam") #SGD


es = EarlyStopping(patience=3)
mc = ModelCheckpoint("best.h5",save_best_only=True)

model.fit(train_generator,validation_data=valid_generator, epochs=100, callbacks=[es,mc])
model.load_weights("best.h5")
result = model.predict(test_generator)

In [ ]:
from sklearn.model_selection import StratifiedKFold #교차 검정 시행
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
result = 0
for train_index,valid_index in skf.split(train,train["invasive"]):
    X_train = train.iloc[train_index]
    X_valid = train.iloc[valid_index]
    idg = ImageDataGenerator()
    idg2 = ImageDataGenerator()
    
    train_generator = idg.flow_from_dataframe()
    valid_generator = idg.flow_from_dataframe()
    test_generator = idg2.flow_from_dataframe()
    
    model = Sequential()
    model.add(EfficientNetB1(weights="imagenet",include_top=False,pooling="avg")) #weight : 다른 곳에서 이미 학습했던 weight를 먼저 가지고 옴(전이 학습) , pooling="avg"는 flatten을 대신해서 정보 손실을 감소
    model.add(Dense(2,activation="softmax"))
    model.compile(metrics=["acc"],loss="categorical_crossentropy",optimizer="adam") #SGD


    es = EarlyStopping(patience=3)
    mc = ModelCheckpoint("best.h5",save_best_only=True)

    model.fit(train_generator,validation_data=valid_generator, epochs=100, callbacks=[es,mc])
    model.load_weights("best.h5")
    result += model.predict(test_generator)/5


In [ ]:
# !unzip /kaggle/input/invasive-species-monitoring/sample_submission.csv.zip

In [ ]:
result

In [ ]:
sub = pd.read_csv("sample_submission.csv")

In [ ]:
sub["name"] = test_generator.filenames
sub["name"] = sub["name"].apply(lambda x:x.split("/")[-1].split('.')[0])
sub["invasive"] = result[:,1]

In [ ]:
sub.to_csv("sub_invasive.csv", index=0)